## Population projections


Notebook environment to migrate TIF files to CF compliant CoG

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# use local or remote data dir
use_local_data = False
ds_dirname = "WP5"

if use_local_data: 
    ds_dir = pathlib.Path().home().joinpath("data", "tmp", ds_dirname)
else: 
    ds_dir = coclico_data_dir.joinpath(ds_dirname)

if not ds_dir.exists():
    raise FileNotFoundError("Directory with data does not exist.")

# directory to export result (make if not exists)
cog_dir = ds_dir.joinpath("single_cog_test") # for checking CF compliancy
cog_dirs = ds_dir.joinpath("cogs") # for making all files CF compliant
cog_dir.mkdir(parents=True, exist_ok=True)
cog_dirs.mkdir(parents=True, exist_ok=True)

C:\Users\soest\AppData\Local\Temp\ipykernel_23536\2770256667.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
test_path = ds_dir.joinpath("pilot","population","SSP1_2010_Europe.tif")
CF_dir = ds_dir.joinpath("CF")

<IPython.core.display.Javascript object>

In [4]:
# NetCDF attribute alterations by means of metadata template
import json
meta_json = open(ds_dir.joinpath("metadata", "metadata_population.json"), 'r')
meta_data = json.load(meta_json)

# NOTE: json module can not handle trailing comma's, these have manually been deleted from the data

<IPython.core.display.Javascript object>

In [5]:
# Run gdal info to check no_data_value shown as _FillValue in xarray
# For this data its: -2147483647
pp_test = xr.open_dataset(test_path, engine="rasterio", mask_and_scale=False) 
pp_test


<xarray.Dataset>
Dimensions:      (band: 1, x: 43200, y: 6507)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 81.85 81.85 81.84 81.83 ... 27.66 27.65 27.65 27.64
    spatial_ref  int32 ...
Data variables:
    band_data    (band, y, x) int32 ...

<IPython.core.display.Javascript object>

In [7]:
# Test for single .tif-file
pp_test = xr.open_dataset(test_path, engine="rasterio", mask_and_scale=False) 
pp_test = pp_test.assign_coords(band=("band", [f"B{k+1:02}" for k in range(pp_test.dims["band"])]))
pp_test = pp_test["band_data"].to_dataset("band")

for var in pp_test:

    da = pp_test[var]

    da = (
    da.where(da != -2147483647, -9999)
    .astype("int32")
    .rio.write_nodata(-9999)
    .rio.set_spatial_dims(x_dim="x", y_dim="y")
    )

    item_name = name_block(
    da,
    storage_prefix="",
    name_prefix="",
    include_band=da.name,
    time_dim=False,
    x_dim="x",
    y_dim="y",
    )

    # convert to dataset
    dad = da.to_dataset()

    # add all attributes (again)
    for attr_name, attr_val in meta_data.items():
        if attr_name == 'PROVIDERS':
            attr_val = json.dumps(attr_val)
        if attr_name == "MEDIA_TYPE": # change media type to tiff, leave the rest as is
            attr_val = "IMAGE/TIFF"
        dad.attrs[attr_name] = attr_val

    dad.attrs['Conventions'] = "CF-1.8"

    # export file
    outpath = cog_dir.joinpath(item_name)
    dad.rio.to_raster(outpath, driver= 'COG')



<IPython.core.display.Javascript object>

In [87]:
# Check newly created .tif

pp_test_new = xr.open_dataset(outpath, engine="rasterio", mask_and_scale=False)

print(np.max(pp_test_new['band_data'].values))
print(np.max(pp_test['B01'].values))

np.size(pp_test['B01'].values)



81283
81283


281102400

In [58]:
# export to nc for quick CF compliancy check..
dad.to_netcdf(path=cog_dir.joinpath(item_name.replace(".tif", ".nc")))

CF_dir

WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/CF')

In [59]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=cog_dir.joinpath(item_name.replace(".tif", ".nc")), working_dir=CF_dir)

In [60]:
# save original CF compliancy
save_compliancy(cap, testfile=cog_dir.joinpath(item_name.replace(".tif", ".nc")), working_dir=CF_dir)

{'P:\\11207608-coclico\\FULLTRACK_DATA\\WP5\\single_cog_test\\B01_x-180.0_y27.64.nc': {'warnings': '5', 'errors': '0'}}
